In [1]:
from IPython.display import Markdown, display
import json
import requests

def print_md(text):
    display(Markdown(text))
def color_md(text, color="blue"):
    return f"<span style='color:{color}'>{text}</span>"

In [2]:

# Retrieve k8s required infos
import subprocess
cmd="bash -i -c 'kSecret-show -n apisix -f admin-token plane-api -v'"
result= subprocess.run(cmd, shell=True, capture_output=True, text=True)
ADMINTOKEN= result.stdout.splitlines()[-1]
print(f"ADMINTOKEN={ADMINTOKEN}")

# cmd="bash -i -c 'kGet -a svc -n apisix control- -o yaml -v  | yq eval '.spec.clusterIP' -"
cmd="bash -i -c 'kGet -a svc -n apisix control- -o yaml -v | yq eval \'.spec.clusterIP\' -'"
result= subprocess.run(cmd, shell=True, capture_output=True, text=True)
output_lines = result.stdout.splitlines()
# print(len(output_lines))
# for o in output_lines:
#     print(o)
IP_APISIXCONTROL= result.stdout.splitlines()[-1]
PORT_APISIXCONTROL=9180
URL_APISIXCONTROL=f"https://{IP_APISIXCONTROL}:{PORT_APISIXCONTROL}"
print(f"APISIXCONTROL_URL={URL_APISIXCONTROL}")
ENDPOINT_ADMINROUTES="apisix/admin/routes"

ADMINTOKEN=DecWcwsv6o1OIQafYp23NNoAnpqL62xs
APISIXCONTROL_URL=https://10.104.23.105:9180


In [3]:
def getRoutes(verbose: True):
    if verbose:
        print_md("## GET routes")
    url=f"{URL_APISIXCONTROL}/{ENDPOINT_ADMINROUTES}"
    # data={"grant_type": "urn:ietf:params:oauth:grant-type:pre-authorized_code",
    #       "pre-authorized_code": PRE_AUTHORIZED_CODE,
    #       "code": PRE_AUTHORIZED_CODE
    # }
    # headers={'Content-Type': 'application/x-www-form-urlencoded'}
    headers={'X-API-KEY': ADMINTOKEN}
    jsonResponse=""
    try:
        response = requests.get(url, headers=headers, verify=False)# , data=data, headers=headers)
        jsonResponse=response.json()
        if verbose:
            print(json.dumps(jsonResponse, indent=4))  # Print the JSON response if it"s in JSON format
        return jsonResponse
        # response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        # CREDENTIAL_ACCESS_TOKEN=jsonResponse["access_token"]
        # print_md (f"{color_md('**CREDENTIAL_ACCESS_TOKEN**', 'green')}=*{CREDENTIAL_ACCESS_TOKEN}*")
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}\n- ErrorResponse:{jsonResponse}\nURL: {url}")# \n- Data: {json.dumps(data, indent=4)}\n- Headers: {json.dumps(headers, indent=4)}")

def createOrUpdateRoute(json_route):
    uri=json_route["uri"]
    host=json_route["host"] if "host" in json_route else None
    x=getRoutes(verbose=False)
    # print(f"creating route {host}:{uri}")
    id=""
    for route in x["list"]:
        uri_=route["value"]["uri"]
        host_=route["value"]["host"] if "host" in route["value"] else None
        # print(f"analyzing route {host_}:{uri_}")
        if uri == uri_ and host == host_:
            id=f'/{route["value"]["id"]}'
            break
    if len(id) == 0:
        verb="POST"
    else:
        verb="PUT"

    print_md(f"## {verb} route {id}")
    url=f"{URL_APISIXCONTROL}/{ENDPOINT_ADMINROUTES}{id}"
    data=json.dumps(json_route)
    # headers={'Content-Type': 'application/x-www-form-urlencoded'}
    headers={'X-API-KEY': ADMINTOKEN, 'Content-Type': 'application/json'}
    jsonResponse=""
    try:
        if len(id) == 0:
            response = requests.post(url, headers=headers, data=data, verify=False)# , data=data, headers=headers)
        else:
            response = requests.put(url, headers=headers, data=data, verify=False)# , data=data, headers=headers)
        jsonResponse=response.json()
        print(json.dumps(jsonResponse, indent=4))  # Print the JSON response if it"s in JSON format
        # response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        # CREDENTIAL_ACCESS_TOKEN=jsonResponse["access_token"]
        # print_md (f"{color_md('**CREDENTIAL_ACCESS_TOKEN**', 'green')}=*{CREDENTIAL_ACCESS_TOKEN}*")
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}\n- ErrorResponse:{jsonResponse}\nURL: {url}\n- Data: {json.dumps(json_route, indent=4)}\n- Headers: {json.dumps(headers, indent=4)}")

def deleteRoute(route_id):
    if not route_id is None:
        print_md(f"## DELETE route {route_id}")
        url=f"{URL_APISIXCONTROL}/{ENDPOINT_ADMINROUTES}/{route_id}"
        # data={"grant_type": "urn:ietf:params:oauth:grant-type:pre-authorized_code",
        #       "pre-authorized_code": PRE_AUTHORIZED_CODE,
        #       "code": PRE_AUTHORIZED_CODE
        # }
        # headers={'Content-Type': 'application/x-www-form-urlencoded'}
        headers={'X-API-KEY': ADMINTOKEN}
        jsonResponse=""
        try:
            response = requests.delete(url, headers=headers, verify=False)# , data=data, headers=headers)
            jsonResponse=response.json()
            print(json.dumps(jsonResponse, indent=4))  # Print the JSON response if it"s in JSON format
            # response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
            # CREDENTIAL_ACCESS_TOKEN=jsonResponse["access_token"]
            # print_md (f"{color_md('**CREDENTIAL_ACCESS_TOKEN**', 'green')}=*{CREDENTIAL_ACCESS_TOKEN}*")
        except requests.exceptions.RequestException as e:
            print(f"Error during request: {e}\n- ErrorResponse:{jsonResponse}\nURL: {url}")# \n- Data: {json.dumps(data, indent=4)}\n- Headers: {json.dumps(headers, indent=4)}")


In [4]:
# https://fiwaredsc-consumer.ita.es/hello
ROUTE_DEMO_JSON={
  "uri": "/hello",
  "host": "fiwaredsc-consumer.local",
  "methods": ["GET"],
  "upstream": {
    "type": "roundrobin",
    "nodes": {
      "echo-svc:8080": 1
    }
  },
  "plugins": {
      "proxy-rewrite": {
          "uri": "/"
      }
  }
}

In [ ]:
getRoutes(verbose=True)
# createOrUpdateRoute(ROUTE_DEMO_JSON)
# deleteRoute("00000000000000000188")


## GET routes

{
    "total": 0,
    "list": []
}


/python/.envs/conda-fiwaredsc/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.104.23.105'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## DELETE route 00000000000000000188

{
    "message": "Key not found"
}


/python/.envs/conda-fiwaredsc/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.104.23.105'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
